# Network Operations

In this notebook we will assemble our project.  We will explore different functions on our dataset and compile them into a workflow ready for production.

The functions we will use will be a mix of `hub` based functions from our [MLRun Functions](http://github.com/mlrun/functions) repo, local and git based notebooks.

> The notebook should be run after generating the data in the [Generator Notebook](generator.ipynb)


we will start by setting up our environment, Loading MLRun and some utilities we will need

In [1]:
# Utils
import os
import json
import urllib
import numpy as np

# MLRun imports
from mlrun import mlconf
import kfp

# Setup API Endpoint
mlconf.dbpath = 'http://mlrun-api:8080'

Now lets define our current project 

## Create a project from a git repository

In [2]:
from mlrun import new_project

# update the dir and repo to reflect real locations 
# the remote git repo must be initialized in GitHub
project_dir = './'
remote_git = 'https://github.com/mlrun/demo-network-operations.git'

# Create the project
newproj = new_project('network-operations', project_dir, init_git=True)

# We can update our project directory to the latest status by running
# newproj.pull()

Now that we have our project directory, lets forword our artifacts there to keep track of them

In [3]:
# Define an artifact path to keep track of where our artifacts are going
ARTIFACT_PATH =  os.path.join(os.path.abspath(newproj.context), 'artifacts')
mlconf.artifact_path = ARTIFACT_PATH

## Create and run functions

As we receive a new dataset, the first thing we would like to do is to explore it a bit, we can do that using our `describe` function in `mlrun/functions`

In [4]:
from mlrun import mount_v3io, new_model_server

In [5]:
# Define function versions to take
describe_tag = 'master'
aggregate_tag = 'development'
train_tag = 'development'
test_tag = 'master'

In [6]:
# Import the functions

# Nuclio fuction from Notebook
newproj.set_function(func='notebooks/generator.ipynb', name='generator')

# Functions From hub
newproj.set_function(func=f'hub://aggregate:{aggregate_tag}', name='aggregate')

newproj.set_function(func=f'hub://describe:{describe_tag}', name='describe')

newproj.set_function(func=f'hub://sklearn_classifier:{train_tag}', name='train')

newproj.set_function(func=f'hub://test_classifier:{test_tag}', name='test')

# Nuclio based Model Server From local notebook
newproj.set_function(func='notebooks/model-server.ipynb', name="serving")

## Generate the dataset
If needed go to [Generator](./generator.ipynb) and run the local workflow to generate the metrics dataset to `data/metrics`

## Run the functions locally to develop the workflow

now we can **Run** the function locally on our sample data, we would like to get some details on our `raw` data

## Register raw data as project level artifact

In [7]:
# Define base Dataset
metrics_path = os.path.join(os.path.abspath(newproj.context), 'data', 'metrics.parquet')
newproj.log_artifact('metrics', target_path = metrics_path)

[mlrun] 2020-04-05 14:23:08,646 log artifact metrics at /User/demo-network-operations/data/metrics.parquet, size: None, db: Y


### Get statistics about the metrics data

In [8]:
from mlrun import NewTask

In [9]:
describe_task = NewTask(
    name="describe", 
    handler="summarize",  
    params={"key": "summary", 
            "label_column": "is_error", 
            'class_labels': ['0', '1'],
            'plot_hist': True,
            'plot_dest': 'plots-metrics'},
    inputs={"table": 'store://network-operations/metrics'},
    artifact_path=ARTIFACT_PATH)

In [10]:
decsribe_run = newproj.func('describe').apply(mount_v3io()).run(describe_task)

[mlrun] 2020-04-05 14:23:44,608 starting run describe uid=7b12525124cf489887a97309f757e457  -> http://mlrun-api:8080
[mlrun] 2020-04-05 14:23:45,189 Job is running in the background, pod: describe-6xllg
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
/opt/conda/lib/python3.7/site-packages/seaborn/distributions.py:288: UserWarning: Data must have variance to compute a kernel density estimate.
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/seaborn/distrib

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
network-operations,...f757e457,0,Apr 05 14:24:34,completed,describe,host=describe-6xllgkind=jobowner=adminv3io_user=admin,table,"class_labels=['0', '1']key=summarylabel_column=is_errorplot_dest=plots-metricsplot_hist=True",scale_pos_weight=11.27,histogramsimbalancecorrelation


to track results use .show() or .logs() or in CLI: 
!mlrun get run 7b12525124cf489887a97309f757e457 --project network-operations , !mlrun logs 7b12525124cf489887a97309f757e457 --project network-operations
[mlrun] 2020-04-05 14:25:59,408 run executed, status=completed


### Create the feature vector

We will use our [Aggregate](https://github.com/mlrun/functions/blob/master/aggregate/aggregate.ipynb) function to create rolling window features for our feature vector.

In doing so we hope that we could help our algorithms identify local errors by using a windowed trend

In [11]:
# Define aggregate task
aggregate_task = NewTask(
    name='aggregate',
    params={'metrics': ['cpu_utilization', 'throughput', 'packet_loss', 'latency'],
            'labels': ['is_error'],
            'metric_aggs': ['mean', 'sum'],
            'label_aggs': ['max'],
            'suffix': 'daily',
            'append_to_df': True,
            'window': 5,
            'center': True,
            'save_to': os.path.join(project_dir, 'data', 'aggregate.pq')},
    inputs={'df_artifact': 'store://network-operations/metrics'},
    handler='aggregate')

In [12]:
aggregate_run = newproj.func('aggregate').apply(mount_v3io()).run(aggregate_task)

[mlrun] 2020-04-05 14:25:59,634 starting run aggregate uid=2adbf8e1e2394d8a86a818812c958b1e  -> http://mlrun-api:8080
[mlrun] 2020-04-05 14:26:00,287 Job is running in the background, pod: aggregate-n6kv2
[mlrun] 2020-04-05 14:26:33,785 Aggregating /User/demo-network-operations/data/metrics.parquet
[mlrun] 2020-04-05 14:26:34,901 log artifact aggregate at /User/demo-network-operations/artifacts/2adbf8e1e2394d8a86a818812c958b1e/./data/aggregate.pq, size: 577194, db: Y

[mlrun] 2020-04-05 14:26:35,108 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
network-operations,...2c958b1e,0,Apr 05 14:26:33,completed,aggregate,host=aggregate-n6kv2kind=jobowner=adminv3io_user=admin,df_artifact,"append_to_df=Truecenter=Truelabel_aggs=['max']labels=['is_error']metric_aggs=['mean', 'sum']metrics=['cpu_utilization', 'throughput', 'packet_loss', 'latency']save_to=./data/aggregate.pqsuffix=dailywindow=5",,aggregate


to track results use .show() or .logs() or in CLI: 
!mlrun get run 2adbf8e1e2394d8a86a818812c958b1e --project network-operations , !mlrun logs 2adbf8e1e2394d8a86a818812c958b1e --project network-operations
[mlrun] 2020-04-05 14:26:41,271 run executed, status=completed


### Get statistics about the feature vector

In [13]:
aggregate_describe_task = NewTask(
    name="describe-aggregate", 
    handler="summarize",  
    params={"key": "summary", 
            "label_column": "is_error", 
            'class_labels': ['0', '1'],
            'plot_hist': True,
            'plot_dest': 'plots-aggregate'},
    inputs={"table": aggregate_run.outputs['aggregate']},
    artifact_path=ARTIFACT_PATH)

In [14]:
aggregate_decsribe_run = newproj.func('describe').apply(mount_v3io()).run(aggregate_describe_task)

[mlrun] 2020-04-05 14:26:41,378 starting run describe-aggregate uid=c4817d2ffcc54854af10c175c5d84817  -> http://mlrun-api:8080
[mlrun] 2020-04-05 14:26:41,969 Job is running in the background, pod: describe-aggregate-66d2j
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
/opt/conda/lib/python3.7/site-packages/seaborn/distributions.py:288: UserWarning: Data must have variance to compute a kernel density estimate.
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-pack

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
network-operations,...c5d84817,0,Apr 05 14:27:26,completed,describe-aggregate,host=describe-aggregate-66d2jkind=jobowner=adminv3io_user=admin,table,"class_labels=['0', '1']key=summarylabel_column=is_errorplot_dest=plots-aggregateplot_hist=True",scale_pos_weight=11.27,histogramsimbalancecorrelation


to track results use .show() or .logs() or in CLI: 
!mlrun get run c4817d2ffcc54854af10c175c5d84817 --project network-operations , !mlrun logs c4817d2ffcc54854af10c175c5d84817 --project network-operations
[mlrun] 2020-04-05 14:33:38,133 run executed, status=completed


## Create workflow to train the model
After reviewing the data and creating the feature vector we move to training our model.  
For this task we will use an **LGBM** classifier.  To control the training process we will supply a `model_config` dictionary with the following parameters:
- **CLASS**: Model-specific parameters.
- **FIT**: Training parameters (like epoch when needed)
- **META**: Model and Package version

### Setup model configurations

In [15]:
model_configs = {
    "CLASS" : {
        "boosting_type"      : "gbdt",
        "num_leaves"         : 300,
        "max_depth"          : 50,
        "learning_rate"      : 0.1,
        "n_estimators"       : 300,
        "objective"          : "binary",
        "scale_pos_weight"   : 1,    
        "min_split_gain"     : 0.0,
        "min_child_samples"  : 20,
        "subsample"          : 1,
        "colsample_bytree"   : 1,
        "reg_alpha"          : 0,
        "reg_lambda"         : 1,
        "n_jobs"             : 16,
        "silent"             : True,
        "importance_type"    : "split",
        "random_state"       : 1},
    "FIT" : {
        "verbose"               : False
    },
    "META" : {
        "class" : "lightgbm.sklearn.LGBMClassifier",
        "version" : "2.3.1"
    }
}
model_config_path = os.path.join(os.path.abspath(newproj.context), 'data', 'lgb_model.json')
with open(model_config_path, 'w') as f:
          f.write(json.dumps(model_configs))

In [16]:
newproj.log_artifact('lgb_configs',
                     target_path = os.path.abspath(model_config_path))

[mlrun] 2020-04-05 14:33:38,454 log artifact lgb_configs at /User/demo-network-operations/data/lgb_model.json, size: None, db: Y


### Create Pipeline Workflow

In [17]:
%%writefile src/workflow.py
from kfp import dsl
from mlrun import mount_v3io
import os

funcs = {}

def init_functions(functions: dict, project=None, secrets=None):
    for f in functions.values():
        # Add V3IO Mount
        f.apply(mount_v3io())
        
        # Always pull images to keep updates
        f.spec.image_pull_policy = 'Always'
                
        
@dsl.pipeline(
    name='Network Operations Demo',
    description='Train a Failure Prediction LGBM Model over sensor data'
)
def kfpipeline(
        df_artifact = 'store://network-operations/metrics',
        metrics = ['cpu_utilization'],
        labels = ['is_error'],
        metric_aggs = ['mean', 'sum'],
        label_aggs = ['max'],
        suffix = 'daily',
        inplace = False,
        window = 5,
        center = True,
        save_to = os.path.join('data', 'aggregate.pq'),
        describe_table = 'summary',
        label_column = 'is_error',
        class_labels = [1, 0],
        SAMPLE_SIZE      = -1, # -n for random sample of n obs, -1 for entire dataset, +n for n consecutive rows
        TEST_SIZE        = 0.1,       # 10% set aside
        TRAIN_VAL_SPLIT  = 0.75,      # remainder split into train and val
        RNG              = 1,
        score_method = 'micro',
        config_filepath = 'store://network-operations/lgb_configs',
    ):
    
    describe = funcs['describe'].as_step(name='describe-raw-data',
                                                handler="summarize",  
                                                params={"key": "summary", 
                                                        "label_column": label_column, 
                                                        'class_labels': ['0', '1'],
                                                        'plot_hist': True,
                                                        'plot_dest': 'plots'},
                                                inputs={"table": df_artifact},
                                                outputs=["summary", "scale_pos_weight"])
    
    # Run preprocessing on the data
    aggregate = funcs['aggregate'].as_step(name='aggregate',
                                                  params={'metrics': metrics,
                                                          'labels': labels,
                                                          'metric_aggs': metric_aggs,
                                                          'label_aggs': label_aggs,
                                                          'suffix': suffix,
                                                          'inplace': inplace,
                                                          'window': window,
                                                          'center': center,
                                                          'save_to': save_to},
                                                  inputs={'df_artifact': df_artifact},
                                                  outputs=['aggregate'],
                                                  handler='aggregate')

    describe = funcs['describe'].as_step(name='describe-feature-vector',
                                                handler="summarize",  
                                                params={"key": "summary", 
                                                        "label_column": label_column, 
                                                        'class_labels': ['0', '1'],
                                                        'plot_hist': True,
                                                        'plot_dest': 'plots'},
                                                inputs={"table": aggregate.outputs['aggregate']},
                                                outputs=["summary", "scale_pos_weight"])
    
    train = funcs['train'].as_step(name='train', 
                                          handler='train_model',
                                          params={'sample'          : SAMPLE_SIZE,
                                                  'label_column'    : label_column,
                                                  'test_size'       : TEST_SIZE,
                                                  'train_val_split' : TRAIN_VAL_SPLIT,
                                                  'rng'             : RNG,
                                                  'score_method'    : score_method},
                                          inputs={"dataset": aggregate.outputs['aggregate'],
                                                  'model_pkg_file' : config_filepath},
                                          outputs=['model', 'test_set'])
    
    test = funcs['test'].as_step(name='test',
                                 handler='test_classifier',
                                 params={'label_column': label_column},
                                 inputs={'models_path': train.outputs['model'],
                                         'test_set': train.outputs['test_set']})
    
    # deploy the model using nuclio functions
    deploy = funcs['serving'].deploy_step(project='nuclio-serving',
                                                 models={'predictor': train.outputs['model']})

Overwriting src/workflow.py


## Add workflow

In [18]:
newproj.set_workflow('main', os.path.join(os.path.abspath(newproj.context), 'src', 'workflow.py'))

## Save Project

In [19]:
newproj.save(os.path.join(newproj.context, 'project.yaml'))

### Run workflow

In [20]:
newproj.run('main', artifact_path=ARTIFACT_PATH, dirty=True)

[mlrun] 2020-04-05 14:33:40,323 WARNING!, you seem to have uncommitted git changes, use .push()


/conda/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['cpu_utilization']".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/conda/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['is_error']".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/conda/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['mean', 'sum']".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/conda/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['

[mlrun] 2020-04-05 14:33:42,379 Pipeline run id=011790ad-3c5c-4c3f-b9e7-540970558086, check UI or DB for progress


'011790ad-3c5c-4c3f-b9e7-540970558086'

## Test endpoint

In [21]:
import pandas as pd
import requests
import json

In [22]:
# Set model
model_name = 'predictor'

# Load pre-processed data example
df = pd.read_parquet(os.path.join(os.path.abspath(newproj.context), 'artifacts', 'data', 'aggregate.pq'))

# Set sample
sample = df.head(1).fillna(0).drop(columns=['is_error']).values.tolist()
msg = {'instances': sample}

# Set endpoint
addr = 'http://192.168.224.209:32666'

In [23]:
# Send Request
req = requests.post(f'{addr}/{model_name}/predict', data=json.dumps(msg))
req.__dict__

{'_content': b'[false]',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Server': 'nuclio', 'Date': 'Sun, 05 Apr 2020 14:33:42 GMT', 'Content-Type': 'application/json', 'Content-Length': '7'},
 'raw': <urllib3.response.HTTPResponse at 0x7f57e5460630>,
 'url': 'http://192.168.224.209:32666/predictor/predict',
 'encoding': None,
 'history': [],
 'reason': 'OK',
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(0, 0, 54624),
 'request': <PreparedRequest [POST]>,
 'connection': <requests.adapters.HTTPAdapter at 0x7f57ec406f60>}